In [ ]:
# ref: https://coolpuppy.readthedocs.io/en/latest/Examples/Walkthrough_API.html

In [ ]:
import os
print(os.getcwd())
os.chdir("/data1/sunwenju/projects_on_940xa/2024041703_mESC_HiC_use_pairtools_cooler_pipeline")
print(os.getcwd())

In [ ]:
! find . -name *rep12.merged.1k.zoomify.mcool

In [ ]:
! find . -name *WT*rep1.*1k.zoomify.mcool

In [ ]:
! find . -name *WT*rep2.*1k.zoomify.mcool

# 读取需要用到的文件

## 读取 mcool 文件

In [ ]:
import cooler
resolution = 10_000
# mcool 文件
nora_wt_mcool_file = "./Nora_WT/Nora_mESC_WT_HiC.rep12.merged.1k.zoomify.mcool"
nora_iaa_mcool_file = "./Nora_IAA/Nora_mESC_IAA_HiC.rep12.merged.1k.zoomify.mcool"
ren_wt_mcool_file = "./Ren_WT/Ren_mESC_WT_HiC.rep12.merged.1k.zoomify.mcool"
ren_iaa_mcool_file = "./Ren_IAA/Ren_mESC_IAA_HiC.rep12.merged.1k.zoomify.mcool"
wn_bpk25_mcool_file = "./WN_BPK25/WN_mESC_BPK25_HiC.rep12.merged.1k.zoomify.mcool"
# 生成 r10k cool 互作矩阵对象
nora_wt_r10k_clr = cooler.Cooler(f'{nora_wt_mcool_file}::/resolutions/{resolution}')
nora_iaa_r10k_clr = cooler.Cooler(f'{nora_iaa_mcool_file}::/resolutions/{resolution}')
ren_wt_r10k_clr = cooler.Cooler(f'{ren_wt_mcool_file}::/resolutions/{resolution}')
ren_iaa_r10k_clr = cooler.Cooler(f'{ren_iaa_mcool_file}::/resolutions/{resolution}')
wn_bpk25_r10k_clr = cooler.Cooler(f'{wn_bpk25_mcool_file}::/resolutions/{resolution}')


In [ ]:
# Nora/Ren WT rep1/2 mcool 后续用于 divide pileup 表示 WT 两次重复之间没啥差异，用 IAA 或 BPK25 处理后有明显差异
nora_wt_rep1_mcool_file = "./Nora_WT/Nora_mESC_WT_HiC_rep1.nodups.pairs.1k.zoomify.mcool"
nora_wt_rep2_mcool_file = "./Nora_WT/Nora_mESC_WT_HiC_rep2.nodups.pairs.1k.zoomify.mcool"
ren_wt_rep1_mcool_file = "./Ren_WT/Ren_mESC_WT_HiC_rep1.nodups.pairs.1k.zoomify.mcool"
ren_wt_rep2_mcool_file = "./Ren_WT/Ren_mESC_WT_HiC_rep2.nodups.pairs.1k.zoomify.mcool"
# 生成 r10k cool 互作矩阵对象
nora_wt_rep1_r10k_clr = cooler.Cooler(f'{nora_wt_rep1_mcool_file}::/resolutions/{resolution}')
nora_wt_rep2_r10k_clr = cooler.Cooler(f'{nora_wt_rep2_mcool_file}::/resolutions/{resolution}')
ren_wt_rep1_r10k_clr = cooler.Cooler(f'{ren_wt_rep1_mcool_file}::/resolutions/{resolution}')
ren_wt_rep2_r10k_clr = cooler.Cooler(f'{ren_wt_rep2_mcool_file}::/resolutions/{resolution}')


## 读取 chrom view 文件

In [ ]:
import pandas as pd
mm10_chr_view_df = pd.read_csv("./mm10.chrom.view.tsv", sep="\t", header=0)
print(mm10_chr_view_df.shape)
mm10_chr_view_df.head()

# 计算生成 expected_cis_df

In [ ]:
from cooltools import expected_cis
# Calculate expected interactions for chromosome arms
nora_wt_r10k_expected_cis_df = expected_cis(clr=nora_wt_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)
nora_iaa_r10k_expected_cis_df = expected_cis(clr=nora_iaa_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)
ren_wt_r10k_expected_cis_df = expected_cis(clr=ren_wt_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)
ren_iaa_r10k_expected_cis_df = expected_cis(clr=ren_iaa_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)
wn_bpk25_r10k_expected_cis_df = expected_cis(clr=wn_bpk25_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)


In [ ]:
# Nora/Ren WT rep1/2 expected_cis_df
nora_wt_rep1_r10k_expected_cis_df = expected_cis(clr=nora_wt_rep1_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)
nora_wt_rep2_r10k_expected_cis_df = expected_cis(clr=nora_wt_rep2_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)
ren_wt_rep1_r10k_expected_cis_df = expected_cis(clr=ren_wt_rep1_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)
ren_wt_rep2_r10k_expected_cis_df = expected_cis(clr=ren_wt_rep2_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)


# 读取 CTCF summit及loops信息并对其进行分类

## 读取 CTCF summit 文件

In [ ]:
import pandas as pd
mESC_WT_CTCF_summit_df = pd.read_csv("./mESC_WT_d0246_union_CTCF.summit.sorted.bed", sep="\t", header=None)
mESC_WT_CTCF_summit_df.columns = ["chrom", "start", "end", "fold_enrichment"]
print(mESC_WT_CTCF_summit_df.shape)
mESC_WT_CTCF_summit_df.head()

In [ ]:
# 为 summit 添加四分类标签
mESC_WT_CTCF_summit_df['quartile_FC_score'] = pd.qcut(mESC_WT_CTCF_summit_df['fold_enrichment'], 4, labels=False) + 1
print(mESC_WT_CTCF_summit_df.shape)
mESC_WT_CTCF_summit_df.head()

In [ ]:
mESC_WT_CTCF_summit_df.quartile_FC_score.value_counts()

## 读取 Nora_WT r10k loops 结果文件

In [ ]:
! find . -name *rep12.merged.mcool_r10k.dots_info.tsv

In [ ]:
nora_wt_r10k_loops_file = "./Nora_WT/Nora_mESC_WT_HiC.rep12.merged.mcool_r10k.dots_info.tsv"
nora_iaa_r10k_loops_file = "./Nora_IAA/Nora_mESC_IAA_HiC.rep12.merged.mcool_r10k.dots_info.tsv"
ren_wt_r10k_loops_file = "./Ren_WT/Ren_mESC_WT_HiC.rep12.merged.mcool_r10k.dots_info.tsv"
ren_iaa_r10k_loops_file = "./Ren_IAA/Ren_mESC_IAA_HiC.rep12.merged.mcool_r10k.dots_info.tsv"
wn_bpk25_r10k_loops_file = "./WN_BPK25/WN_mESC_BPK25_HiC.rep12.merged.mcool_r10k.dots_info.tsv"
nora_wt_r10k_loops_df = pd.read_csv(nora_wt_r10k_loops_file, sep="\t", header=0)
print(nora_wt_r10k_loops_df.shape)
nora_wt_r10k_loops_df.head()

In [ ]:
nora_wt_r10k_loops_df.describe()

## 提取 loops 有用列

In [ ]:
usecols = ["chrom1","start1","end1","chrom2","start2","end2","count"]
nora_wt_r10k_loops_df = nora_wt_r10k_loops_df[usecols]
nora_wt_r10k_loops_df["count_quartile"] = pd.qcut(nora_wt_r10k_loops_df['count'], 4, labels=False) + 1
print(nora_wt_r10k_loops_df.shape)
nora_wt_r10k_loops_df.head()


In [ ]:
nora_wt_r10k_loops_df.count_quartile.value_counts()

## 给 loop bedpe 添加 CTCF BS 分类列

In [ ]:
# 自定义函数判断 loop 两个anchor 是否有 CTCF 结合， 已与 bedtools intersect 结果比较，数量一致
def add_CBS_type_for_loops_bedpe(loop_bedpe_row, ctcf_bs_bed, ext_len=25):
    chrom1, start1, end1 = loop_bedpe_row["chrom1"], loop_bedpe_row["start1"], loop_bedpe_row["end1"]
    chrom2, start2, end2 = loop_bedpe_row["chrom2"], loop_bedpe_row["start2"], loop_bedpe_row["end2"]
    CBS_type = []
    # CBS_type: 
    # N 表示两侧anchor都没CTCF结合
    # L 表示loop左侧anchor有CTCF结合
    # R 表示loop右侧anchor有CTCF结合
    # B 表示loop两侧anchor都有CTCF结合
    for bed_row in ctcf_bs_bed.itertuples(index=True, name='Pandas'):
        cbs_chr = bed_row.chrom
        cbs_start = bed_row.start - ext_len
        cbs_end = bed_row.end + ext_len
        if chrom1 == cbs_chr and (start1<=cbs_start<end1 or start1<cbs_end<=end1):
            CBS_type.append("L")
            break
    for bed_row in ctcf_bs_bed.itertuples(index=True, name='Pandas'):
        cbs_chr = bed_row.chrom
        cbs_start = bed_row.start - ext_len
        cbs_end = bed_row.end + ext_len
        if chrom2 == cbs_chr and (start2<=cbs_start<end2 or start2<cbs_end<=end2):
            CBS_type.append("R")
            break
    if "L" in CBS_type and "R" in CBS_type:
        loop_bedpe_row_cbs_type = "B"
    elif CBS_type == ["L"]:
        loop_bedpe_row_cbs_type = "L"
    elif CBS_type == ["R"]:
        loop_bedpe_row_cbs_type = "R"
    else:
        loop_bedpe_row_cbs_type = "N"
    return loop_bedpe_row_cbs_type

nora_wt_r10k_loops_df["CBS_type"] = nora_wt_r10k_loops_df.apply(add_CBS_type_for_loops_bedpe, axis=1, args=(mESC_WT_CTCF_summit_df, 25))
nora_wt_r10k_loops_df.head()


In [ ]:
import numpy as np
nora_wt_r10k_loops_df['CTCF_BS'] = np.where(nora_wt_r10k_loops_df['CBS_type'] == "N", 'No', 'Yes')
nora_wt_r10k_loops_df['loop_len'] = nora_wt_r10k_loops_df['start2'] - nora_wt_r10k_loops_df['start1']
nora_wt_r10k_loops_df.head()

In [ ]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
nora_wt_r10k_loops_df.describe()

In [ ]:
nora_wt_r10k_loops_df.CBS_type.value_counts()

In [ ]:
nora_wt_r10k_loops_df[nora_wt_r10k_loops_df["loop_len"]>=200000].CBS_type.value_counts()

In [ ]:
nora_wt_r10k_loops_df.CTCF_BS.value_counts()

In [ ]:
nora_wt_r10k_loops_df[nora_wt_r10k_loops_df["loop_len"]>=200000].CTCF_BS.value_counts()


# Nora_WT r10k loops 不分组 pileup

## Nora_WT

In [ ]:
from coolpuppy import coolpup
nora_wt_r10k_clr_loops_pup = coolpup.pileup(clr=nora_wt_r10k_clr, 
                                            features=nora_wt_r10k_loops_df,
                                            features_format='bedpe', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_wt_r10k_expected_cis_df,
                                            flank=100_000,
                                            min_diag=0,
                                            nproc=32)
nora_wt_r10k_clr_loops_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wt_r10k_clr_loops_pup,
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Nora_WT.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Nora_WT.png")

## Nora_WT_rep1

In [ ]:
from coolpuppy import coolpup
nora_wt_rep1_r10k_clr_loops_pup = coolpup.pileup(clr=nora_wt_rep1_r10k_clr, 
                                            features=nora_wt_r10k_loops_df,
                                            features_format='bedpe', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_wt_rep1_r10k_expected_cis_df,
                                            flank=100_000,
                                            min_diag=0,
                                            nproc=32)
nora_wt_rep1_r10k_clr_loops_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wt_rep1_r10k_clr_loops_pup,
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Nora_WT_rep1.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Nora_WT_rep1.png")

## Nora_WT_rep2

In [ ]:
from coolpuppy import coolpup
nora_wt_rep2_r10k_clr_loops_pup = coolpup.pileup(clr=nora_wt_rep2_r10k_clr, 
                                            features=nora_wt_r10k_loops_df,
                                            features_format='bedpe', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_wt_rep2_r10k_expected_cis_df,
                                            flank=100_000,
                                            min_diag=0,
                                            nproc=32)
nora_wt_rep2_r10k_clr_loops_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wt_rep2_r10k_clr_loops_pup,
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Nora_WT_rep2.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Nora_WT_rep2.png")

## Nora_IAA

In [ ]:
from coolpuppy import coolpup
nora_iaa_r10k_clr_loops_pup = coolpup.pileup(clr=nora_iaa_r10k_clr, 
                                            features=nora_wt_r10k_loops_df,
                                            features_format='bedpe', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_iaa_r10k_expected_cis_df,
                                            flank=100_000,
                                            min_diag=0,
                                            nproc=32)
nora_iaa_r10k_clr_loops_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_iaa_r10k_clr_loops_pup,
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Nora_IAA.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Nora_IAA.png")

## Ren_WT

In [ ]:
from coolpuppy import coolpup
ren_wt_r10k_clr_loops_pup = coolpup.pileup(clr=ren_wt_r10k_clr, 
                                            features=nora_wt_r10k_loops_df,
                                            features_format='bedpe', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_wt_r10k_expected_cis_df,
                                            flank=100_000,
                                            min_diag=0,
                                            nproc=32)
ren_wt_r10k_clr_loops_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_wt_r10k_clr_loops_pup,
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Ren_WT.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Ren_WT.png")

## Ren_WT_rep1

In [ ]:
from coolpuppy import coolpup
ren_wt_rep1_r10k_clr_loops_pup = coolpup.pileup(clr=ren_wt_rep1_r10k_clr, 
                                            features=nora_wt_r10k_loops_df,
                                            features_format='bedpe', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_wt_rep1_r10k_expected_cis_df,
                                            flank=100_000,
                                            min_diag=0,
                                            nproc=32)
ren_wt_rep1_r10k_clr_loops_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_wt_rep1_r10k_clr_loops_pup,
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Ren_WT_rep1.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Ren_WT_rep1.png")

## Ren_WT_rep2

In [ ]:
from coolpuppy import coolpup
ren_wt_rep2_r10k_clr_loops_pup = coolpup.pileup(clr=ren_wt_rep2_r10k_clr, 
                                            features=nora_wt_r10k_loops_df,
                                            features_format='bedpe', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_wt_rep2_r10k_expected_cis_df,
                                            flank=100_000,
                                            min_diag=0,
                                            nproc=32)
ren_wt_rep2_r10k_clr_loops_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_wt_rep2_r10k_clr_loops_pup,
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Ren_WT_rep2.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Ren_WT_rep2.png")

## Ren_IAA

In [ ]:
from coolpuppy import coolpup
ren_iaa_r10k_clr_loops_pup = coolpup.pileup(clr=ren_iaa_r10k_clr, 
                                            features=nora_wt_r10k_loops_df,
                                            features_format='bedpe', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_iaa_r10k_expected_cis_df,
                                            flank=100_000,
                                            min_diag=0,
                                            nproc=32)
ren_iaa_r10k_clr_loops_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_iaa_r10k_clr_loops_pup,
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Ren_IAA.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Ren_IAA.png")

## WN_BPK25

In [ ]:
from coolpuppy import coolpup
wn_bpk25_r10k_clr_loops_pup = coolpup.pileup(clr=wn_bpk25_r10k_clr, 
                                            features=nora_wt_r10k_loops_df,
                                            features_format='bedpe', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=wn_bpk25_r10k_expected_cis_df,
                                            flank=100_000,
                                            min_diag=0,
                                            nproc=32)
wn_bpk25_r10k_clr_loops_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=wn_bpk25_r10k_clr_loops_pup,
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.WN_BPK25.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.WN_BPK25.png")

## Nora_WT_IAA_WN_BPK25_merge_in_one

In [ ]:
# nora_wt_r10k_clr_loops_pup
# nora_iaa_r10k_clr_loops_pup
# wn_bpk25_r10k_clr_loops_pup
nora_wn_loops_pup_merged_df = pd.concat([nora_wt_r10k_clr_loops_pup, nora_iaa_r10k_clr_loops_pup, wn_bpk25_r10k_clr_loops_pup], ignore_index=True)
nora_wn_loops_pup_merged_df


In [ ]:
nora_wn_loops_pup_merged_df["group"] = ["Nora_WT", "Nora_IAA", "WN_BPK25"]
nora_wn_loops_pup_merged_df

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wn_loops_pup_merged_df,
                cols="group",
                col_order=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                colnames=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Nora_WN.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Nora_WN.png")


## Ren_WT_IAA_WN_BPK25_merge_in_one

In [ ]:
# ren_wt_r10k_clr_loops_pup
# ren_iaa_r10k_clr_loops_pup
# wn_bpk25_r10k_clr_loops_pup
ren_wn_loops_pup_merged_df = pd.concat([ren_wt_r10k_clr_loops_pup, ren_iaa_r10k_clr_loops_pup, wn_bpk25_r10k_clr_loops_pup], ignore_index=True)
ren_wn_loops_pup_merged_df


In [ ]:
ren_wn_loops_pup_merged_df["group"] = ["Ren_WT", "Ren_IAA", "WN_BPK25"]
ren_wn_loops_pup_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_wn_loops_pup_merged_df,
                cols="group",
                col_order=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                colnames=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Ren_WN.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Ren_WN.png")


## all experiment merge in one

In [ ]:
nora_wn_ren_loops_pup_merged_df = pd.concat([nora_wt_r10k_clr_loops_pup, nora_iaa_r10k_clr_loops_pup, wn_bpk25_r10k_clr_loops_pup, ren_iaa_r10k_clr_loops_pup, ren_wt_r10k_clr_loops_pup], ignore_index=True)
nora_wn_ren_loops_pup_merged_df


In [ ]:
nora_wn_ren_loops_pup_merged_df["group"] = ["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"]
nora_wn_ren_loops_pup_merged_df

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wn_ren_loops_pup_merged_df,
                cols="group",
                col_order=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                colnames=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                score=True,
                vmax=5,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Nora_WN_Ren.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Nora_WN_Ren.png")


## Nora_WT_rep2_vs_rep1

In [ ]:
# nora_wt_rep1_r10k_clr_loops_pup
# nora_wt_rep2_r10k_clr_loops_pup
from coolpuppy.lib.puputils import divide_pups
loops_pup_nora_wt_rep2_vs_rep1 = divide_pups(nora_wt_rep2_r10k_clr_loops_pup, nora_wt_rep1_r10k_clr_loops_pup)
loops_pup_nora_wt_rep2_vs_rep1


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_pup_nora_wt_rep2_vs_rep1,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.Nora_WT_rep2_vs_rep1.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.Nora_WT_rep2_vs_rep1.png")


## Ren_WT_rep2_vs_rep1

In [ ]:
# ren_wt_rep1_r10k_clr_loops_pup
# ren_wt_rep2_r10k_clr_loops_pup
from coolpuppy.lib.puputils import divide_pups
loops_pup_ren_wt_rep2_vs_rep1 = divide_pups(ren_wt_rep2_r10k_clr_loops_pup, ren_wt_rep1_r10k_clr_loops_pup)
loops_pup_ren_wt_rep2_vs_rep1


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_pup_ren_wt_rep2_vs_rep1,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.Ren_WT_rep2_vs_rep1.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.Ren_WT_rep2_vs_rep1.png")


## Nora_IAA_vs_WT

In [ ]:
# nora_wt_r10k_clr_loops_pup
# nora_iaa_r10k_clr_loops_pup
from coolpuppy.lib.puputils import divide_pups
loops_pup_nora_iaa_vs_wt = divide_pups(nora_iaa_r10k_clr_loops_pup, nora_wt_r10k_clr_loops_pup)
loops_pup_nora_iaa_vs_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_pup_nora_iaa_vs_wt,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.Nora_IAA_vs_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.Nora_IAA_vs_WT.png")


## Ren_IAA_vs_WT

In [ ]:
# ren_wt_r10k_clr_loops_pup
# ren_iaa_r10k_clr_loops_pup
from coolpuppy.lib.puputils import divide_pups
loops_pup_ren_iaa_vs_wt = divide_pups(ren_iaa_r10k_clr_loops_pup, ren_wt_r10k_clr_loops_pup)
loops_pup_ren_iaa_vs_wt

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_pup_ren_iaa_vs_wt,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.Ren_IAA_vs_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.Ren_IAA_vs_WT.png")

## WN_BPK25_vs_Nora_WT

In [ ]:
# wn_bpk25_r10k_clr_loops_pup
# nora_wt_r10k_clr_loops_pup
from coolpuppy.lib.puputils import divide_pups
loops_pup_wn_bpk25_vs_nora_wt = divide_pups(wn_bpk25_r10k_clr_loops_pup, nora_wt_r10k_clr_loops_pup)
loops_pup_wn_bpk25_vs_nora_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_pup_wn_bpk25_vs_nora_wt,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.WN_BPK25_vs_Nora_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.WN_BPK25_vs_Nora_WT.png")


## WN_BPK25_vs_Ren_WT

In [ ]:
# wn_bpk25_r10k_clr_loops_pup
# ren_wt_r10k_clr_loops_pup
from coolpuppy.lib.puputils import divide_pups
loops_pup_wn_bpk25_vs_ren_wt = divide_pups(wn_bpk25_r10k_clr_loops_pup, ren_wt_r10k_clr_loops_pup)
loops_pup_wn_bpk25_vs_ren_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_pup_wn_bpk25_vs_ren_wt,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.WN_BPK25_vs_Ren_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.WN_BPK25_vs_Ren_WT.png")


## merge 4 divide pileup in one

In [ ]:
loops_pup_divide_merged_df = pd.concat([loops_pup_nora_iaa_vs_wt, loops_pup_ren_iaa_vs_wt, loops_pup_wn_bpk25_vs_nora_wt, loops_pup_wn_bpk25_vs_ren_wt], ignore_index=True)
loops_pup_divide_merged_df

In [ ]:
loops_pup_divide_merged_df.insert(0, "IAA_vs_WT", ["Yes", "Yes", "No", "No"] )
loops_pup_divide_merged_df.insert(1, "WT_source", ["Nora", "Ren", "Nora", "Ren"] )
loops_pup_divide_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_pup_divide_merged_df,
                rows="IAA_vs_WT",
                row_order=["Yes", "No"],
                rownames=["IAA_vs_WT", "BPK25_vs_WT"],
                cols='WT_source',
                col_order=["Nora", "Ren"],
                colnames=["use_Nora_WT", "use_Ren_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.merge_4comparation.exp_col.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.merge_4comparation.exp_col.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_pup_divide_merged_df,
                cols="IAA_vs_WT",
                col_order=["Yes", "No"],
                colnames=["IAA_vs_WT", "BPK25_vs_WT"],
                rows='WT_source',
                row_order=["Nora", "Ren"],
                rownames=["use_Nora_WT", "use_Ren_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.merge_4comparation.exp_row.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.merge_4comparation.exp_row.png")


## merge 6 divide pileup in one

In [ ]:
loops_pup_divide_merged_df = pd.concat([loops_pup_nora_wt_rep2_vs_rep1, loops_pup_ren_wt_rep2_vs_rep1, loops_pup_nora_iaa_vs_wt, loops_pup_ren_iaa_vs_wt, loops_pup_wn_bpk25_vs_nora_wt, loops_pup_wn_bpk25_vs_ren_wt], ignore_index=True)
loops_pup_divide_merged_df

In [ ]:
loops_pup_divide_merged_df.insert(0, "comp_group", ["WT_rep2vs1", "WT_rep2vs1", "IAA_vs_WT", "IAA_vs_WT", "BPK25_vs_WT", "BPK25_vs_WT"] )
loops_pup_divide_merged_df.insert(1, "WT_source", ["Nora", "Ren", "Nora", "Ren", "Nora", "Ren"] )
loops_pup_divide_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_pup_divide_merged_df,
                rows="comp_group",
                row_order=["WT_rep2vs1", "IAA_vs_WT", "BPK25_vs_WT"],
                rownames=["WT_rep2vs1", "IAA_vs_WT", "BPK25_vs_WT"],
                cols='WT_source',
                col_order=["Nora", "Ren"],
                colnames=["use_Nora_WT", "use_Ren_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.merge_6comparation.exp_col.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.merge_6comparation.exp_col.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_pup_divide_merged_df,
                cols="comp_group",
                col_order=["WT_rep2vs1", "IAA_vs_WT", "BPK25_vs_WT"],
                colnames=["WT_rep2vs1", "IAA_vs_WT", "BPK25_vs_WT"],
                rows='WT_source',
                row_order=["Nora", "Ren"],
                rownames=["use_Nora_WT", "use_Ren_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.merge_6comparation.exp_row.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.merge_6comparation.exp_row.png")


## merge plot for Nora WN 3 comparation

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_pup_divide_merged_df,
                cols="comp_group",
                col_order=["WT_rep2vs1", "IAA_vs_WT", "BPK25_vs_WT"],
                colnames=["WT_rep2vs1", "IAA_vs_WT", "BPK25_vs_WT"],
                rows='WT_source',
                row_order=["Nora"],
                rownames=["use_Nora_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.merge_Nora_WN_3comparation.exp_row.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.merge_Nora_WN_3comparation.exp_row.png")


## merge plot for Ren WN 3 comparation

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_pup_divide_merged_df,
                cols="comp_group",
                col_order=["WT_rep2vs1", "IAA_vs_WT", "BPK25_vs_WT"],
                colnames=["WT_rep2vs1", "IAA_vs_WT", "BPK25_vs_WT"],
                rows='WT_source',
                row_order=["Ren"],
                rownames=["use_Ren_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.merge_Ren_WN_3comparation.exp_row.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.merge_Ren_WN_3comparation.exp_row.png")


# Nora_WT r10k loops 根据 CBS 分两组 pileup

## Nora_WT

In [ ]:
from coolpuppy import coolpup
nora_wt_r10k_clr_loops_CBS2c_pup = coolpup.pileup(clr=nora_wt_r10k_clr, 
                                            features=nora_wt_r10k_loops_df,
                                            features_format='bedpe', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_wt_r10k_expected_cis_df,
                                            groupby=["CTCF_BS"],
                                            flank=100_000,
                                            min_diag=0,
                                            nproc=32)
nora_wt_r10k_clr_loops_CBS2c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_loops_count = nora_wt_r10k_loops_df.shape[0]
CBS_loop_count = nora_wt_r10k_loops_df.CTCF_BS.value_counts()["Yes"]
nonCBS_loop_count = nora_wt_r10k_loops_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=nora_wt_r10k_clr_loops_CBS2c_pup,
                cols="CTCF_BS",
                col_order=["all", "Yes", "No"],
                colnames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                vmax=7,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WT.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WT.png")


## Nora_WT_rep1

In [ ]:
from coolpuppy import coolpup
nora_wt_rep1_r10k_clr_loops_CBS2c_pup = coolpup.pileup(clr=nora_wt_rep1_r10k_clr, 
                                            features=nora_wt_r10k_loops_df,
                                            features_format='bedpe', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_wt_rep1_r10k_expected_cis_df,
                                            groupby=["CTCF_BS"],
                                            flank=100_000,
                                            min_diag=0,
                                            nproc=32)
nora_wt_rep1_r10k_clr_loops_CBS2c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_loops_count = nora_wt_r10k_loops_df.shape[0]
CBS_loop_count = nora_wt_r10k_loops_df.CTCF_BS.value_counts()["Yes"]
nonCBS_loop_count = nora_wt_r10k_loops_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=nora_wt_rep1_r10k_clr_loops_CBS2c_pup,
                cols="CTCF_BS",
                col_order=["all", "Yes", "No"],
                colnames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                vmax=6,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WT_rep1.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WT_rep1.png")


## Nora_WT_rep2

In [ ]:
from coolpuppy import coolpup
nora_wt_rep2_r10k_clr_loops_CBS2c_pup = coolpup.pileup(clr=nora_wt_rep2_r10k_clr, 
                                            features=nora_wt_r10k_loops_df,
                                            features_format='bedpe', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_wt_rep2_r10k_expected_cis_df,
                                            groupby=["CTCF_BS"],
                                            flank=100_000,
                                            min_diag=0,
                                            nproc=32)
nora_wt_rep2_r10k_clr_loops_CBS2c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_loops_count = nora_wt_r10k_loops_df.shape[0]
CBS_loop_count = nora_wt_r10k_loops_df.CTCF_BS.value_counts()["Yes"]
nonCBS_loop_count = nora_wt_r10k_loops_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=nora_wt_rep2_r10k_clr_loops_CBS2c_pup,
                cols="CTCF_BS",
                col_order=["all", "Yes", "No"],
                colnames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                vmax=7,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WT_rep2.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WT_rep2.png")


## Nora_IAA

In [ ]:
from coolpuppy import coolpup
nora_iaa_r10k_clr_loops_CBS2c_pup = coolpup.pileup(clr=nora_iaa_r10k_clr, 
                                            features=nora_wt_r10k_loops_df,
                                            features_format='bedpe', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_iaa_r10k_expected_cis_df,
                                            groupby=["CTCF_BS"],
                                            flank=100_000,
                                            min_diag=0,
                                            nproc=32)
nora_iaa_r10k_clr_loops_CBS2c_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_loops_count = nora_wt_r10k_loops_df.shape[0]
CBS_loop_count = nora_wt_r10k_loops_df.CTCF_BS.value_counts()["Yes"]
nonCBS_loop_count = nora_wt_r10k_loops_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=nora_iaa_r10k_clr_loops_CBS2c_pup,
                cols="CTCF_BS",
                col_order=["all", "Yes", "No"],
                colnames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                vmax=7,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_IAA.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_IAA.png")


## Ren_WT

In [ ]:
from coolpuppy import coolpup
ren_wt_r10k_clr_loops_CBS2c_pup = coolpup.pileup(clr=ren_wt_r10k_clr, 
                                            features=nora_wt_r10k_loops_df,
                                            features_format='bedpe', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_wt_r10k_expected_cis_df,
                                            groupby=["CTCF_BS"],
                                            flank=100_000,
                                            min_diag=0,
                                            nproc=32)
ren_wt_r10k_clr_loops_CBS2c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_loops_count = nora_wt_r10k_loops_df.shape[0]
CBS_loop_count = nora_wt_r10k_loops_df.CTCF_BS.value_counts()["Yes"]
nonCBS_loop_count = nora_wt_r10k_loops_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=ren_wt_r10k_clr_loops_CBS2c_pup,
                cols="CTCF_BS",
                col_order=["all", "Yes", "No"],
                colnames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_WT.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_WT.png")


## Ren_WT_rep1

In [ ]:
from coolpuppy import coolpup
ren_wt_rep1_r10k_clr_loops_CBS2c_pup = coolpup.pileup(clr=ren_wt_rep1_r10k_clr, 
                                            features=nora_wt_r10k_loops_df,
                                            features_format='bedpe', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_wt_rep1_r10k_expected_cis_df,
                                            groupby=["CTCF_BS"],
                                            flank=100_000,
                                            min_diag=0,
                                            nproc=32)
ren_wt_rep1_r10k_clr_loops_CBS2c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_loops_count = nora_wt_r10k_loops_df.shape[0]
CBS_loop_count = nora_wt_r10k_loops_df.CTCF_BS.value_counts()["Yes"]
nonCBS_loop_count = nora_wt_r10k_loops_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=ren_wt_rep1_r10k_clr_loops_CBS2c_pup,
                cols="CTCF_BS",
                col_order=["all", "Yes", "No"],
                colnames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_WT_rep1.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_WT_rep1.png")


## Ren_WT_rep2

In [ ]:
from coolpuppy import coolpup
ren_wt_rep2_r10k_clr_loops_CBS2c_pup = coolpup.pileup(clr=ren_wt_rep2_r10k_clr, 
                                            features=nora_wt_r10k_loops_df,
                                            features_format='bedpe', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_wt_rep2_r10k_expected_cis_df,
                                            groupby=["CTCF_BS"],
                                            flank=100_000,
                                            min_diag=0,
                                            nproc=32)
ren_wt_rep2_r10k_clr_loops_CBS2c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_loops_count = nora_wt_r10k_loops_df.shape[0]
CBS_loop_count = nora_wt_r10k_loops_df.CTCF_BS.value_counts()["Yes"]
nonCBS_loop_count = nora_wt_r10k_loops_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=ren_wt_rep2_r10k_clr_loops_CBS2c_pup,
                cols="CTCF_BS",
                col_order=["all", "Yes", "No"],
                colnames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_WT_rep2.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_WT_rep2.png")


## Ren_IAA

In [ ]:
from coolpuppy import coolpup
ren_iaa_r10k_clr_loops_CBS2c_pup = coolpup.pileup(clr=ren_iaa_r10k_clr, 
                                            features=nora_wt_r10k_loops_df,
                                            features_format='bedpe', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_iaa_r10k_expected_cis_df,
                                            groupby=["CTCF_BS"],
                                            flank=100_000,
                                            min_diag=0,
                                            nproc=32)
ren_iaa_r10k_clr_loops_CBS2c_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_loops_count = nora_wt_r10k_loops_df.shape[0]
CBS_loop_count = nora_wt_r10k_loops_df.CTCF_BS.value_counts()["Yes"]
nonCBS_loop_count = nora_wt_r10k_loops_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=ren_iaa_r10k_clr_loops_CBS2c_pup,
                cols="CTCF_BS",
                col_order=["all", "Yes", "No"],
                colnames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_IAA.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_IAA.png")


## WN_BPK25

In [ ]:
from coolpuppy import coolpup
wn_bpk25_r10k_clr_loops_CBS2c_pup = coolpup.pileup(clr=wn_bpk25_r10k_clr, 
                                            features=nora_wt_r10k_loops_df,
                                            features_format='bedpe', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=wn_bpk25_r10k_expected_cis_df,
                                            groupby=["CTCF_BS"],
                                            flank=100_000,
                                            min_diag=0,
                                            nproc=32)
wn_bpk25_r10k_clr_loops_CBS2c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_loops_count = nora_wt_r10k_loops_df.shape[0]
CBS_loop_count = nora_wt_r10k_loops_df.CTCF_BS.value_counts()["Yes"]
nonCBS_loop_count = nora_wt_r10k_loops_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=wn_bpk25_r10k_clr_loops_CBS2c_pup,
                cols="CTCF_BS",
                col_order=["all", "Yes", "No"],
                colnames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                vmax=5,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.WN_BPK25.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.WN_BPK25.png")

## Nora_WT_IAA_WN_BPK25_merge_in_one

In [ ]:
loops_CBS2c_pup_nora_wn_merged_df = pd.concat([nora_wt_r10k_clr_loops_CBS2c_pup, nora_iaa_r10k_clr_loops_CBS2c_pup, wn_bpk25_r10k_clr_loops_CBS2c_pup], ignore_index=True)
loops_CBS2c_pup_nora_wn_merged_df


In [ ]:
loops_CBS2c_pup_nora_wn_merged_df["group"] = ["Nora_WT"]*3 + ["Nora_IAA"]*3 + ["WN_BPK25"]*3
loops_CBS2c_pup_nora_wn_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_CBS2c_pup_nora_wn_merged_df,
                rows="group",
                row_order=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                rownames=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                cols="CTCF_BS",
                col_order=["all", "Yes", "No"],
                colnames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                vmax=7,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WN.exp_row.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WN.exp_row.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_CBS2c_pup_nora_wn_merged_df,
                cols="group",
                col_order=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                colnames=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                rows="CTCF_BS",
                row_order=["all", "Yes", "No"],
                rownames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                vmax=7,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WN.exp_col.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WN.exp_col.png")


## Ren_WT_IAA_WN_BPK25_merge_in_one

In [ ]:
loops_CBS2c_pup_ren_wn_merged_df = pd.concat([ren_wt_r10k_clr_loops_CBS2c_pup, ren_iaa_r10k_clr_loops_CBS2c_pup, wn_bpk25_r10k_clr_loops_CBS2c_pup], ignore_index=True)
loops_CBS2c_pup_ren_wn_merged_df


In [ ]:
loops_CBS2c_pup_ren_wn_merged_df["group"] = ["Ren_WT"]*3 + ["Ren_IAA"]*3 + ["WN_BPK25"]*3
loops_CBS2c_pup_ren_wn_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_CBS2c_pup_ren_wn_merged_df,
                rows="group",
                row_order=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                rownames=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                cols="CTCF_BS",
                col_order=["all", "Yes", "No"],
                colnames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_WN.exp_row.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_WN.exp_row.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_CBS2c_pup_ren_wn_merged_df,
                cols="group",
                col_order=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                colnames=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                rows="CTCF_BS",
                row_order=["all", "Yes", "No"],
                rownames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_WN.exp_col.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_WN.exp_col.png")


## all experiment merge in one

In [ ]:
loops_CBS2c_pup_nora_wn_ren_merged_df = pd.concat([nora_wt_r10k_clr_loops_CBS2c_pup, nora_iaa_r10k_clr_loops_CBS2c_pup, wn_bpk25_r10k_clr_loops_CBS2c_pup, ren_iaa_r10k_clr_loops_CBS2c_pup, ren_wt_r10k_clr_loops_CBS2c_pup], ignore_index=True)
loops_CBS2c_pup_nora_wn_ren_merged_df


In [ ]:
loops_CBS2c_pup_nora_wn_ren_merged_df["group"] = ["Nora_WT"]*3 + ["Nora_IAA"]*3 + ["WN_BPK25"]*3 + ["Ren_IAA"]*3 + ["Ren_WT"]*3
loops_CBS2c_pup_nora_wn_ren_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_CBS2c_pup_nora_wn_ren_merged_df,
                rows="group",
                row_order=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                rownames=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                cols="CTCF_BS",
                col_order=["all", "Yes", "No"],
                colnames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                vmax=7,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WN_Ren.exp_row.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WN_Ren.exp_row.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_CBS2c_pup_nora_wn_ren_merged_df,
                cols="group",
                col_order=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                colnames=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                rows="CTCF_BS",
                row_order=["all", "Yes", "No"],
                rownames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                vmax=7,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WN_Ren.exp_col.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WN_Ren.exp_col.png")


## Nora_WT_rep2_vs_rep1

In [ ]:
# nora_wt_rep1_r10k_clr_loops_CBS2c_pup
# nora_wt_rep2_r10k_clr_loops_CBS2c_pup
from coolpuppy.lib.puputils import divide_pups
all_CTCF_BS_group_divide_pup = []
for CTCF_BS_group in ["all", "Yes", "No"]:
    rep1_pup = nora_wt_rep1_r10k_clr_loops_CBS2c_pup[nora_wt_rep1_r10k_clr_loops_CBS2c_pup.CTCF_BS==CTCF_BS_group]
    rep2_pup = nora_wt_rep2_r10k_clr_loops_CBS2c_pup[nora_wt_rep2_r10k_clr_loops_CBS2c_pup.CTCF_BS==CTCF_BS_group]
    rep2_vs_rep1 = divide_pups(rep2_pup, rep1_pup)
    all_CTCF_BS_group_divide_pup.append(rep2_vs_rep1)
loops_CBS2c_pup_nora_wt_rep2_vs_rep1 = pd.concat(all_CTCF_BS_group_divide_pup, ignore_index=True)
loops_CBS2c_pup_nora_wt_rep2_vs_rep1


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_CBS2c_pup_nora_wt_rep2_vs_rep1,
            cols='CTCF_BS',
            col_order=["all", "Yes", "No"],
            colnames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WT_rep2_vs_rep1.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WT_rep2_vs_rep1.png")


## Ren_WT_rep2_vs_rep1

In [ ]:
# ren_wt_rep1_r10k_clr_loops_CBS2c_pup
# ren_wt_rep2_r10k_clr_loops_CBS2c_pup
from coolpuppy.lib.puputils import divide_pups
all_CTCF_BS_group_divide_pup = []
for CTCF_BS_group in ["all", "Yes", "No"]:
    rep1_pup = ren_wt_rep1_r10k_clr_loops_CBS2c_pup[ren_wt_rep1_r10k_clr_loops_CBS2c_pup.CTCF_BS==CTCF_BS_group]
    rep2_pup = ren_wt_rep2_r10k_clr_loops_CBS2c_pup[ren_wt_rep2_r10k_clr_loops_CBS2c_pup.CTCF_BS==CTCF_BS_group]
    rep2_vs_rep1 = divide_pups(rep2_pup, rep1_pup)
    all_CTCF_BS_group_divide_pup.append(rep2_vs_rep1)
loops_CBS2c_pup_ren_wt_rep2_vs_rep1 = pd.concat(all_CTCF_BS_group_divide_pup, ignore_index=True)
loops_CBS2c_pup_ren_wt_rep2_vs_rep1


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_CBS2c_pup_ren_wt_rep2_vs_rep1,
            cols='CTCF_BS',
            col_order=["all", "Yes", "No"],
            colnames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_WT_rep2_vs_rep1.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_WT_rep2_vs_rep1.png")


## Nora_IAA_vs_WT

In [ ]:
# nora_wt_r10k_clr_loops_CBS2c_pup
# nora_iaa_r10k_clr_loops_CBS2c_pup
from coolpuppy.lib.puputils import divide_pups
all_CTCF_BS_group_divide_pup = []
for CTCF_BS_group in ["all", "Yes", "No"]:
    iaa_pup = nora_iaa_r10k_clr_loops_CBS2c_pup[nora_iaa_r10k_clr_loops_CBS2c_pup.CTCF_BS==CTCF_BS_group]
    wt_pup = nora_wt_r10k_clr_loops_CBS2c_pup[nora_wt_r10k_clr_loops_CBS2c_pup.CTCF_BS==CTCF_BS_group]
    iaa_vs_wt = divide_pups(iaa_pup, wt_pup)
    all_CTCF_BS_group_divide_pup.append(iaa_vs_wt)
loops_CBS2c_pup_nora_iaa_vs_wt = pd.concat(all_CTCF_BS_group_divide_pup, ignore_index=True)
loops_CBS2c_pup_nora_iaa_vs_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_CBS2c_pup_nora_iaa_vs_wt,
            cols='CTCF_BS',
            col_order=["all", "Yes", "No"],
            colnames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_IAA_vs_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_IAA_vs_WT.png")

## Ren_IAA_vs_WT

In [ ]:
# ren_wt_r10k_clr_loops_CBS2c_pup
# ren_iaa_r10k_clr_loops_CBS2c_pup
from coolpuppy.lib.puputils import divide_pups
all_CTCF_BS_group_divide_pup = []
for CTCF_BS_group in ["all", "Yes", "No"]:
    iaa_pup = ren_iaa_r10k_clr_loops_CBS2c_pup[ren_iaa_r10k_clr_loops_CBS2c_pup.CTCF_BS==CTCF_BS_group]
    wt_pup = ren_wt_r10k_clr_loops_CBS2c_pup[ren_wt_r10k_clr_loops_CBS2c_pup.CTCF_BS==CTCF_BS_group]
    iaa_vs_wt = divide_pups(iaa_pup, wt_pup)
    all_CTCF_BS_group_divide_pup.append(iaa_vs_wt)
loops_CBS2c_pup_ren_iaa_vs_wt = pd.concat(all_CTCF_BS_group_divide_pup, ignore_index=True)
loops_CBS2c_pup_ren_iaa_vs_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_CBS2c_pup_ren_iaa_vs_wt,
            cols='CTCF_BS',
            col_order=["all", "Yes", "No"],
            colnames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_IAA_vs_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_IAA_vs_WT.png")


## WN_BPK25_vs_Nora_WT

In [ ]:
# nora_wt_r10k_clr_loops_CBS2c_pup
# wn_bpk25_r10k_clr_loops_CBS2c_pup
from coolpuppy.lib.puputils import divide_pups
all_CTCF_BS_group_divide_pup = []
for CTCF_BS_group in ["all", "Yes", "No"]:
    wn_bpk25_pup = wn_bpk25_r10k_clr_loops_CBS2c_pup[wn_bpk25_r10k_clr_loops_CBS2c_pup.CTCF_BS==CTCF_BS_group]
    nora_wt_pup = nora_wt_r10k_clr_loops_CBS2c_pup[nora_wt_r10k_clr_loops_CBS2c_pup.CTCF_BS==CTCF_BS_group]
    wn_bpk25_vs_nora_wt = divide_pups(wn_bpk25_pup, nora_wt_pup)
    all_CTCF_BS_group_divide_pup.append(wn_bpk25_vs_nora_wt)
loops_CBS2c_pup_wn_bpk25_vs_nora_wt = pd.concat(all_CTCF_BS_group_divide_pup, ignore_index=True)
loops_CBS2c_pup_wn_bpk25_vs_nora_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_CBS2c_pup_wn_bpk25_vs_nora_wt,
            cols='CTCF_BS',
            col_order=["all", "Yes", "No"],
            colnames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.WN_BPK25_vs_Nora_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.WN_BPK25_vs_Nora_WT.png")

## WN_BPK25_vs_Ren_WT

In [ ]:
# ren_wt_r10k_clr_loops_CBS2c_pup
# wn_bpk25_r10k_clr_loops_CBS2c_pup
from coolpuppy.lib.puputils import divide_pups
all_CTCF_BS_group_divide_pup = []
for CTCF_BS_group in ["all", "Yes", "No"]:
    wn_bpk25_pup = wn_bpk25_r10k_clr_loops_CBS2c_pup[wn_bpk25_r10k_clr_loops_CBS2c_pup.CTCF_BS==CTCF_BS_group]
    ren_wt_pup = ren_wt_r10k_clr_loops_CBS2c_pup[ren_wt_r10k_clr_loops_CBS2c_pup.CTCF_BS==CTCF_BS_group]
    wn_bpk25_vs_ren_wt = divide_pups(wn_bpk25_pup, ren_wt_pup)
    all_CTCF_BS_group_divide_pup.append(wn_bpk25_vs_ren_wt)
loops_CBS2c_pup_wn_bpk25_vs_ren_wt = pd.concat(all_CTCF_BS_group_divide_pup, ignore_index=True)
loops_CBS2c_pup_wn_bpk25_vs_ren_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_CBS2c_pup_wn_bpk25_vs_ren_wt,
            cols='CTCF_BS',
            col_order=["all", "Yes", "No"],
            colnames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.WN_BPK25_vs_Ren_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.WN_BPK25_vs_Ren_WT.png")


## merge 4 divide pileup in one

In [ ]:
loops_CBS2c_pup_divide_merged_df = pd.concat([loops_CBS2c_pup_nora_iaa_vs_wt, loops_CBS2c_pup_ren_iaa_vs_wt, loops_CBS2c_pup_wn_bpk25_vs_nora_wt, loops_CBS2c_pup_wn_bpk25_vs_ren_wt], ignore_index=True)
loops_CBS2c_pup_divide_merged_df

In [ ]:
loops_CBS2c_pup_divide_merged_df.insert(0, "divide_group", ["Nora_IAA_vs_WT"]*3+["Ren_IAA_vs_WT"]*3+["WN_BPK25_vs_Nora_WT"]*3+["WN_BPK25_vs_Ren_WT"]*3)
loops_CBS2c_pup_divide_merged_df

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_CBS2c_pup_divide_merged_df,
                rows="divide_group",
                row_order=["Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                rownames=["Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                cols='CTCF_BS',
                col_order=["all", "Yes", "No"],
                colnames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.merge_4comparation.exp_row.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.merge_4comparation.exp_row.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_CBS2c_pup_divide_merged_df,
                cols="divide_group",
                col_order=["Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                colnames=["Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                rows='CTCF_BS',
                row_order=["all", "Yes", "No"],
                rownames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.merge_4comparation.exp_col.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.merge_4comparation.exp_col.png")


## merge 6 divide pileup in one

In [ ]:
loops_CBS2c_pup_divide_merged_df = pd.concat([loops_CBS2c_pup_nora_wt_rep2_vs_rep1, loops_CBS2c_pup_ren_wt_rep2_vs_rep1, loops_CBS2c_pup_nora_iaa_vs_wt, loops_CBS2c_pup_ren_iaa_vs_wt, loops_CBS2c_pup_wn_bpk25_vs_nora_wt, loops_CBS2c_pup_wn_bpk25_vs_ren_wt], ignore_index=True)
loops_CBS2c_pup_divide_merged_df


In [ ]:
loops_CBS2c_pup_divide_merged_df.insert(0, "divide_group", ["Nora_WT_rep2vs1"]*3+["Ren_WT_rep2vs1"]*3+["Nora_IAA_vs_WT"]*3+["Ren_IAA_vs_WT"]*3+["WN_BPK25_vs_Nora_WT"]*3+["WN_BPK25_vs_Ren_WT"]*3)
loops_CBS2c_pup_divide_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_CBS2c_pup_divide_merged_df,
                rows="divide_group",
                row_order=["Nora_WT_rep2vs1", "Ren_WT_rep2vs1", "Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                rownames=["Nora_WT_rep2vs1", "Ren_WT_rep2vs1", "Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                cols='CTCF_BS',
                col_order=["all", "Yes", "No"],
                colnames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.merge_6comparation.exp_row.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.merge_6comparation.exp_row.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_CBS2c_pup_divide_merged_df,
                cols="divide_group",
                col_order=["Nora_WT_rep2vs1", "Ren_WT_rep2vs1", "Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                colnames=["Nora_WT_rep2vs1", "Ren_WT_rep2vs1", "Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                rows='CTCF_BS',
                row_order=["all", "Yes", "No"],
                rownames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.merge_6comparation.exp_col.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.merge_6comparation.exp_col.png")


## merge plot for Nora WN 3 comparation

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_CBS2c_pup_divide_merged_df,
                cols="divide_group",
                col_order=["Nora_WT_rep2vs1", "Nora_IAA_vs_WT", "WN_BPK25_vs_Nora_WT"],
                colnames=["Nora_WT_rep2vs1", "Nora_IAA_vs_WT", "WN_BPK25_vs_Nora_WT"],
                rows='CTCF_BS',
                row_order=["all", "Yes", "No"],
                rownames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.merge_Nora_WN_3comparation.exp_col.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.merge_Nora_WN_3comparation.exp_col.png")


## merge plot for Ren WN 3 comparation

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=loops_CBS2c_pup_divide_merged_df,
                cols="divide_group",
                col_order=["Ren_WT_rep2vs1", "Ren_IAA_vs_WT", "WN_BPK25_vs_Ren_WT"],
                colnames=["Ren_WT_rep2vs1", "Ren_IAA_vs_WT", "WN_BPK25_vs_Ren_WT"],
                rows='CTCF_BS',
                row_order=["all", "Yes", "No"],
                rownames=[f'Nora_WT_all_loops\n({all_loops_count})', f'loops_with_CBS\n({CBS_loop_count})', f'loops_without_CBS\n({nonCBS_loop_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.merge_Ren_WN_3comparation.exp_col.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.merge_Ren_WN_3comparation.exp_col.png")


# 制作结果汇总 PPT

In [ ]:
! ls -1 coolpuppy_*for_Nora_WT_r10k_loops*.pdf

In [ ]:
! ls -1 coolpuppy_*for_Nora_WT_r10k_loops*.png

In [ ]:
from PIL import Image
from pptx import Presentation
from pptx.util import Pt, Cm
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN

nora_wt_r10k_loops_noC_Nora_WT_rep1_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Nora_WT_rep1.png"
nora_wt_r10k_loops_noC_Nora_WT_rep2_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Nora_WT_rep2.png"
nora_wt_r10k_loops_noC_Ren_WT_rep1_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Ren_WT_rep1.png"
nora_wt_r10k_loops_noC_Ren_WT_rep2_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Ren_WT_rep2.png"

nora_wt_r10k_loops_noC_Nora_WT_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Nora_WT.png"
nora_wt_r10k_loops_noC_Nora_IAA_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Nora_IAA.png"
nora_wt_r10k_loops_noC_Ren_WT_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Ren_WT.png"
nora_wt_r10k_loops_noC_Ren_IAA_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Ren_IAA.png"
nora_wt_r10k_loops_noC_WN_BPK25_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.WN_BPK25.png"

nora_wt_r10k_loops_noC_Nora_WN_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Nora_WN.png"
nora_wt_r10k_loops_noC_Ren_WN_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Ren_WN.png"
nora_wt_r10k_loops_noC_Nora_WN_Ren_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_noClassified.Nora_WN_Ren.png"

def add_text_anno(slide, anno_text, left, top, width, height):
    # 插入文本框
    text_box = slide.shapes.add_textbox(left, top, width, height)
    # 设置文本框背景颜色为黄色
    text_box.fill.solid()
    text_box.fill.fore_color.rgb = RGBColor(255, 230, 153)
    # 在文本框中添加内容并设置格式
    text_box.text_frame.text = anno_text
    text_box.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
    # 设置字体大小和加粗
    text_font = text_box.text_frame.paragraphs[0].font
    text_font.name = '微软雅黑'
    text_font.size = Pt(6)
    text_font.bold = True


# 创建 PPT 并插入图片
my_ppt = Presentation()
# 设置幻灯片尺寸为 16:9 (25.4cm *14.28cm)
my_ppt.slide_width = 9144000
my_ppt.slide_height = 5143500

# slide1
slide1 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
# pic = slide.shapes.add_picture(png_file, left, top, width, height)
slide1.shapes.add_picture(nora_wt_r10k_loops_noC_Nora_WT_rep1_png, Cm(3.21), Cm(0), Cm(7.14), Cm(7.14))
slide1.shapes.add_picture(nora_wt_r10k_loops_noC_Nora_WT_rep2_png, Cm(15.07), Cm(0), Cm(7.14), Cm(7.14))
slide1.shapes.add_picture(nora_wt_r10k_loops_noC_Ren_WT_rep1_png, Cm(3.21), Cm(7.14), Cm(7.14), Cm(7.14))
slide1.shapes.add_picture(nora_wt_r10k_loops_noC_Ren_WT_rep2_png, Cm(15.07), Cm(7.14), Cm(7.14), Cm(7.14))
# 插入文本注释
add_text_anno(slide1, "nora_wt_r10k_loops_noC_Nora_WT_rep1_pileup_plot", Cm(3.21), Cm(0.05), Cm(7.14), Cm(0.5))
add_text_anno(slide1, "nora_wt_r10k_loops_noC_Nora_WT_rep2_pileup_plot", Cm(15.07), Cm(0.05), Cm(7.14), Cm(0.5))
add_text_anno(slide1, "nora_wt_r10k_loops_noC_Ren_WT_rep1_pileup_plot", Cm(3.21), Cm(7.19), Cm(7.14), Cm(0.5))
add_text_anno(slide1, "nora_wt_r10k_loops_noC_Ren_WT_rep2_pileup_plot", Cm(15.07), Cm(7.19), Cm(7.14), Cm(0.5))

# slide2
slide2 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide2.shapes.add_picture(nora_wt_r10k_loops_noC_Nora_WT_png, Cm(0.7), Cm(0.05), Cm(7.125), Cm(7.125))
slide2.shapes.add_picture(nora_wt_r10k_loops_noC_Nora_IAA_png, Cm(9.14), Cm(0.05), Cm(7.125), Cm(7.125))
slide2.shapes.add_picture(nora_wt_r10k_loops_noC_WN_BPK25_png, Cm(17.625), Cm(0.05), Cm(7.125), Cm(7.125))
slide2.shapes.add_picture(nora_wt_r10k_loops_noC_Ren_WT_png, Cm(0.7), Cm(7.14), Cm(7.125), Cm(7.125))
slide2.shapes.add_picture(nora_wt_r10k_loops_noC_Ren_IAA_png, Cm(9.14), Cm(7.14), Cm(7.125), Cm(7.125))
# 插入文本注释
add_text_anno(slide2, "nora_wt_r10k_loops_noC_Nora_WT_pileup_plot", Cm(0.1), Cm(0.05), Cm(8), Cm(0.5))
add_text_anno(slide2, "nora_wt_r10k_loops_noC_Nora_IAA_pileup_plot", Cm(8.65), Cm(0.05), Cm(8), Cm(0.5))
add_text_anno(slide2, "nora_wt_r10k_loops_noC_WN_BPK25_pileup_plot", Cm(17.3), Cm(0.05), Cm(8), Cm(0.5))
add_text_anno(slide2, "nora_wt_r10k_loops_noC_Ren_WT_pileup_plot", Cm(0.1), Cm(7.24), Cm(8), Cm(0.5))
add_text_anno(slide2, "nora_wt_r10k_loops_noC_Ren_IAA_pileup_plot", Cm(8.65), Cm(7.24), Cm(8), Cm(0.5))

# slide3
slide3 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide3.shapes.add_picture(nora_wt_r10k_loops_noC_Nora_WN_png, Cm(6.23), Cm(0.02), Cm(12.94), Cm(4.31))
slide3.shapes.add_picture(nora_wt_r10k_loops_noC_Ren_WN_png, Cm(6.23), Cm(4.54),  Cm(12.94), Cm(4.31))
slide3.shapes.add_picture(nora_wt_r10k_loops_noC_Nora_WN_Ren_png, Cm(0), Cm(9.04), Cm(25.4), Cm(5.08))
# 插入文本注释
add_text_anno(slide3, "nora_wt_r10k_loops_noC_Nora_WN_pileup_plot", Cm(0.28), Cm(1.92), Cm(7), Cm(0.5))
add_text_anno(slide3, "nora_wt_r10k_loops_noC_Ren_WN_pileup_plot", Cm(0.28), Cm(6.44), Cm(7), Cm(0.5))
add_text_anno(slide3, "nora_wt_r10k_loops_noC_Nora_WN_pileup_plot", Cm(0.28), Cm(8.86), Cm(7), Cm(0.5))

# 保存 PPT
my_ppt.save('8.3_coolpuppy_pileup_plot_for_Nora_WT_r10k_loop_anchors.use_Nora_WN_Ren_mcool.r10k.summary.pptx')


In [ ]:
noraWT_r10k_loops_noC_nora_rep2vs1 = "coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.Nora_WT_rep2_vs_rep1.png"
noraWT_r10k_loops_noC_nora_iaa_vs_wt = "coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.Nora_IAA_vs_WT.png"
noraWT_r10k_loops_noC_wn_bpk25_vs_nora_wt = "coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.WN_BPK25_vs_Nora_WT.png"

noraWT_r10k_loops_noC_ren_rep2vs1 = "coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.Ren_WT_rep2_vs_rep1.png"
noraWT_r10k_loops_noC_ren_iaa_vs_wt = "coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.Ren_IAA_vs_WT.png"
noraWT_r10k_loops_noC_wn_bpk25_vs_ren_wt = "coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.WN_BPK25_vs_Ren_WT.png"

# 打开 PPT 并插入图片
my_ppt = Presentation('8.3_coolpuppy_pileup_plot_for_Nora_WT_r10k_loop_anchors.use_Nora_WN_Ren_mcool.r10k.summary.pptx')
# slide4
slide4 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
# pic = slide.shapes.add_picture(png_files[pi-1], left, top, width, height)
slide4.shapes.add_picture(noraWT_r10k_loops_noC_nora_rep2vs1, Cm(0.53), Cm(0), Cm(7.14), Cm(7.14))
slide4.shapes.add_picture(noraWT_r10k_loops_noC_nora_iaa_vs_wt, Cm(9.08), Cm(0), Cm(7.14), Cm(7.14))
slide4.shapes.add_picture(noraWT_r10k_loops_noC_wn_bpk25_vs_nora_wt, Cm(17.73), Cm(0), Cm(7.14), Cm(7.14))
slide4.shapes.add_picture(noraWT_r10k_loops_noC_ren_rep2vs1, Cm(0.53), Cm(7.14), Cm(7.14), Cm(7.14))
slide4.shapes.add_picture(noraWT_r10k_loops_noC_ren_iaa_vs_wt, Cm(9.08), Cm(7.14), Cm(7.14), Cm(7.14))
slide4.shapes.add_picture(noraWT_r10k_loops_noC_wn_bpk25_vs_ren_wt, Cm(17.73), Cm(7.14), Cm(7.14), Cm(7.14))
# 插入文本注释
add_text_anno(slide4, "noraWT_r10k_loops_noC_nora_rep2vs1", Cm(0.53), Cm(0), Cm(7.14), Cm(0.5))
add_text_anno(slide4, "noraWT_r10k_loops_noC_nora_iaa_vs_wt", Cm(9.08), Cm(0), Cm(7.14), Cm(0.5))
add_text_anno(slide4, "noraWT_r10k_loops_noC_wn_bpk25_vs_nora_wt", Cm(17.73), Cm(0), Cm(7.14), Cm(0.5))
add_text_anno(slide4, "noraWT_r10k_loops_noC_ren_rep2vs1", Cm(0.53), Cm(7.14), Cm(7.14), Cm(0.5))
add_text_anno(slide4, "noraWT_r10k_loops_noC_ren_iaa_vs_wt", Cm(9.08), Cm(7.14), Cm(7.14), Cm(0.5))
add_text_anno(slide4, "noraWT_r10k_loops_noC_wn_bpk25_vs_ren_wt", Cm(17.73), Cm(7.14), Cm(7.14), Cm(0.5))

noraWT_r10k_loops_noC_nora_wn_3comparation = "coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.merge_Nora_WN_3comparation.exp_row.png"
noraWT_r10k_loops_noC_ren_wn_3comparation = "coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.merge_Ren_WN_3comparation.exp_row.png"
noraWT_r10k_loops_noC_nora_wn_ren_6comparation = "coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_noClassified.merge_6comparation.exp_row.png"
# slide5
slide5 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
# pic = slide.shapes.add_picture(png_files[pi-1], left, top, width, height)
slide5.shapes.add_picture(noraWT_r10k_loops_noC_nora_wn_3comparation, Cm(1.98), Cm(0), Cm(21.43), Cm(7.14))
slide5.shapes.add_picture(noraWT_r10k_loops_noC_ren_wn_3comparation, Cm(1.98), Cm(7.14), Cm(21.43), Cm(7.14))
# 插入文本注释
add_text_anno(slide5, "noraWT_r10k_loops_noC_nora_wn_3comparation", Cm(0), Cm(0.05), Cm(12.7), Cm(0.5))
add_text_anno(slide5, "noraWT_r10k_loops_noC_ren_wn_3comparation", Cm(0), Cm(7.14), Cm(12.7), Cm(0.5))

# slide6
slide6 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide6.shapes.add_picture(noraWT_r10k_loops_noC_nora_wn_ren_6comparation, Cm(1.98), Cm(0), Cm(21.43), Cm(14.28))
# 插入文本注释
add_text_anno(slide6, "noraWT_r10k_loops_noC_nora_wn_ren_6comparation", Cm(0), Cm(0.05), Cm(12.7), Cm(0.5))

# 保存 PPT
my_ppt.save('8.3_coolpuppy_pileup_plot_for_Nora_WT_r10k_loop_anchors.use_Nora_WN_Ren_mcool.r10k.summary.pptx')


In [ ]:
# 打开 PPT 并插入图片
my_ppt = Presentation('8.3_coolpuppy_pileup_plot_for_Nora_WT_r10k_loop_anchors.use_Nora_WN_Ren_mcool.r10k.summary.pptx')
# slide7
slide7 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])

nora_wt_r10k_loops_CBS2c_Nora_WT_rep1_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WT_rep1.png"
nora_wt_r10k_loops_CBS2c_Nora_WT_rep2_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WT_rep2.png"
nora_wt_r10k_loops_CBS2c_Ren_WT_rep1_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_WT_rep1.png"
nora_wt_r10k_loops_CBS2c_Ren_WT_rep2_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_WT_rep2.png"
# slide8
slide8 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
# pic = slide.shapes.add_picture(png_files[pi-1], left, top, width, height)
slide8.shapes.add_picture(nora_wt_r10k_loops_CBS2c_Nora_WT_rep1_png, Cm(0), Cm(0.59), Cm(12.7), Cm(4.23))
slide8.shapes.add_picture(nora_wt_r10k_loops_CBS2c_Nora_WT_rep2_png, Cm(0), Cm(5.38), Cm(12.7), Cm(4.23))
slide8.shapes.add_picture(nora_wt_r10k_loops_CBS2c_Ren_WT_rep1_png, Cm(12.7), Cm(0.59), Cm(12.7), Cm(4.23))
slide8.shapes.add_picture(nora_wt_r10k_loops_CBS2c_Ren_WT_rep2_png, Cm(12.7), Cm(5.38), Cm(12.7), Cm(4.23))
# 插入文本注释
add_text_anno(slide8, "nora_wt_r10k_loops_CBS2c_Nora_WT_rep1_png", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide8, "nora_wt_r10k_loops_CBS2c_Nora_WT_rep2_png", Cm(0), Cm(4.85), Cm(10.85), Cm(0.5))
add_text_anno(slide8, "nora_wt_r10k_loops_CBS2c_Ren_WT_rep1_png", Cm(14.55), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide8, "nora_wt_r10k_loops_CBS2c_Ren_WT_rep2_png", Cm(14.55), Cm(4.85), Cm(10.85), Cm(0.5))

nora_wt_r10k_loops_CBS2c_Nora_WT_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WT.png"
nora_wt_r10k_loops_CBS2c_Nora_IAA_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_IAA.png"
nora_wt_r10k_loops_CBS2c_Ren_WT_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_WT.png"
nora_wt_r10k_loops_CBS2c_Ren_IAA_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_IAA.png"
nora_wt_r10k_loops_CBS2c_WN_BPK25_png = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.WN_BPK25.png"
# slide9
slide9 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide9.shapes.add_picture(nora_wt_r10k_loops_CBS2c_Nora_WT_png, Cm(0), Cm(0.59), Cm(12.7), Cm(4.23))
slide9.shapes.add_picture(nora_wt_r10k_loops_CBS2c_Nora_IAA_png, Cm(0), Cm(5.38), Cm(12.7), Cm(4.23))
slide9.shapes.add_picture(nora_wt_r10k_loops_CBS2c_WN_BPK25_png, Cm(0), Cm(10.06), Cm(12.7), Cm(4.23))
slide9.shapes.add_picture(nora_wt_r10k_loops_CBS2c_Ren_WT_png, Cm(12.7), Cm(0.59), Cm(12.7), Cm(4.23))
slide9.shapes.add_picture(nora_wt_r10k_loops_CBS2c_Ren_IAA_png, Cm(12.7), Cm(5.38), Cm(12.7), Cm(4.23))
# 插入文本注释
add_text_anno(slide9, "nora_wt_r10k_loops_CBS2c_Nora_WT_png", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide9, "nora_wt_r10k_loops_CBS2c_Nora_IAA_png", Cm(0), Cm(4.85), Cm(10.85), Cm(0.5))
add_text_anno(slide9, "nora_wt_r10k_loops_CBS2c_WN_BPK25_png", Cm(0), Cm(9.53), Cm(10.85), Cm(0.5))
add_text_anno(slide9, "nora_wt_r10k_loops_CBS2c_Ren_WT_png", Cm(14.55), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide9, "nora_wt_r10k_loops_CBS2c_Ren_IAA_png", Cm(14.55), Cm(4.85), Cm(10.85), Cm(0.5))

# 保存 PPT
my_ppt.save('8.3_coolpuppy_pileup_plot_for_Nora_WT_r10k_loop_anchors.use_Nora_WN_Ren_mcool.r10k.summary.pptx')


In [ ]:
# 打开 PPT 并插入图片
my_ppt = Presentation('8.3_coolpuppy_pileup_plot_for_Nora_WT_r10k_loop_anchors.use_Nora_WN_Ren_mcool.r10k.summary.pptx')

nora_wt_r10k_loops_CBS2c_Nora_WN_row = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WN.exp_row.png"
nora_wt_r10k_loops_CBS2c_Ren_WN_row = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_WN.exp_row.png"
# slide10
slide10 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide10.shapes.add_picture(nora_wt_r10k_loops_CBS2c_Nora_WN_row, Cm(0), Cm(0.79), Cm(12.7), Cm(12.7))
slide10.shapes.add_picture(nora_wt_r10k_loops_CBS2c_Ren_WN_row, Cm(12.7), Cm(0.79), Cm(12.7), Cm(12.7))
# 插入文本注释
add_text_anno(slide10, "nora_wt_r10k_loops_CBS2c_Nora_WN_row", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide10, "nora_wt_r10k_loops_CBS2c_Ren_WN_row", Cm(14.55), Cm(0.05), Cm(10.85), Cm(0.5))

nora_wt_r10k_loops_CBS2c_Nora_WN_col = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WN.exp_col.png"
nora_wt_r10k_loops_CBS2c_Ren_WN_col = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_WN.exp_col.png"
# slide11
slide11 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide11.shapes.add_picture(nora_wt_r10k_loops_CBS2c_Nora_WN_col, Cm(0), Cm(0.79), Cm(12.7), Cm(12.7))
slide11.shapes.add_picture(nora_wt_r10k_loops_CBS2c_Ren_WN_col, Cm(12.7), Cm(0.79), Cm(12.7), Cm(12.7))
# 插入文本注释
add_text_anno(slide11, "nora_wt_r10k_loops_CBS2c_Nora_WN_col", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide11, "nora_wt_r10k_loops_CBS2c_Ren_WN_col", Cm(14.55), Cm(0.05), Cm(10.85), Cm(0.5))

nora_wt_r10k_loops_CBS2c_Nora_WN_Ren = "coolpuppy_pileup_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WN_Ren.exp_col.png"
# slide12
slide12 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide12.shapes.add_picture(nora_wt_r10k_loops_CBS2c_Nora_WN_Ren, Cm(0.79), Cm(0), Cm(23.81), Cm(14.28))
# 插入文本注释
add_text_anno(slide12, "nora_wt_r10k_loops_CBS2c_Nora_WN_Ren", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))

# 保存 PPT
my_ppt.save('8.3_coolpuppy_pileup_plot_for_Nora_WT_r10k_loop_anchors.use_Nora_WN_Ren_mcool.r10k.summary.pptx')


In [ ]:
# 打开 PPT 并插入图片
my_ppt = Presentation('8.3_coolpuppy_pileup_plot_for_Nora_WT_r10k_loop_anchors.use_Nora_WN_Ren_mcool.r10k.summary.pptx')

nora_wt_r10k_loops_CBS2c_Nora_rep2vs1 = "coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_WT_rep2_vs_rep1.png"
nora_wt_r10k_loops_CBS2c_Nora_IAA_vs_WT = "coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.Nora_IAA_vs_WT.png"
nora_wt_r10k_loops_CBS2c_WN_BPK25_vs_Nora_WT = "coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.WN_BPK25_vs_Nora_WT.png"
nora_wt_r10k_loops_CBS2c_Ren_rep2vs1 = "coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_WT_rep2_vs_rep1.png"
nora_wt_r10k_loops_CBS2c_Ren_IAA_vs_WT = "coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.Ren_IAA_vs_WT.png"
nora_wt_r10k_loops_CBS2c_WN_BPK25_vs_Ren_WT = "coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.WN_BPK25_vs_Ren_WT.png"
# slide13
slide13 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide13.shapes.add_picture(nora_wt_r10k_loops_CBS2c_Nora_rep2vs1, Cm(0), Cm(0.59), Cm(12.7), Cm(4.23))
slide13.shapes.add_picture(nora_wt_r10k_loops_CBS2c_Nora_IAA_vs_WT, Cm(0), Cm(5.38), Cm(12.7), Cm(4.23))
slide13.shapes.add_picture(nora_wt_r10k_loops_CBS2c_WN_BPK25_vs_Nora_WT, Cm(0), Cm(10.06), Cm(12.7), Cm(4.23))
slide13.shapes.add_picture(nora_wt_r10k_loops_CBS2c_Ren_rep2vs1, Cm(12.7), Cm(0.59), Cm(12.7), Cm(4.23))
slide13.shapes.add_picture(nora_wt_r10k_loops_CBS2c_Ren_IAA_vs_WT, Cm(12.7), Cm(5.38), Cm(12.7), Cm(4.23))
slide13.shapes.add_picture(nora_wt_r10k_loops_CBS2c_WN_BPK25_vs_Ren_WT, Cm(12.7), Cm(10.06), Cm(12.7), Cm(4.23))
# 插入文本注释
add_text_anno(slide13, "nora_wt_r10k_loops_CBS2c_Nora_rep2vs1", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide13, "nora_wt_r10k_loops_CBS2c_Nora_IAA_vs_WT", Cm(0), Cm(4.85), Cm(10.85), Cm(0.5))
add_text_anno(slide13, "nora_wt_r10k_loops_CBS2c_WN_BPK25_vs_Nora_WT", Cm(0), Cm(9.53), Cm(10.85), Cm(0.5))
add_text_anno(slide13, "nora_wt_r10k_loops_CBS2c_Ren_rep2vs1", Cm(14.55), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide13, "nora_wt_r10k_loops_CBS2c_Ren_IAA_vs_WT", Cm(14.55), Cm(4.85), Cm(10.85), Cm(0.5))
add_text_anno(slide13, "nora_wt_r10k_loops_CBS2c_WN_BPK25_vs_Ren_WT", Cm(14.55), Cm(9.53), Cm(10.85), Cm(0.5))

# 保存 PPT
my_ppt.save('8.3_coolpuppy_pileup_plot_for_Nora_WT_r10k_loop_anchors.use_Nora_WN_Ren_mcool.r10k.summary.pptx')


In [ ]:
# 打开 PPT 并插入图片
my_ppt = Presentation('8.3_coolpuppy_pileup_plot_for_Nora_WT_r10k_loop_anchors.use_Nora_WN_Ren_mcool.r10k.summary.pptx')

nora_wt_r10k_loops_CBS2c_divide_Nora_WN_col = "coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.merge_Nora_WN_3comparation.exp_col.png"
nora_wt_r10k_loops_CBS2c_divide_Ren_WN_col = "coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.merge_Ren_WN_3comparation.exp_col.png"
# slide14
slide14 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide14.shapes.add_picture(nora_wt_r10k_loops_CBS2c_divide_Nora_WN_col, Cm(0), Cm(0.79), Cm(12.7), Cm(12.7))
slide14.shapes.add_picture(nora_wt_r10k_loops_CBS2c_divide_Ren_WN_col, Cm(12.7), Cm(0.79), Cm(12.7), Cm(12.7))
# 插入文本注释
add_text_anno(slide14, "nora_wt_r10k_loops_CBS2c_divide_Nora_WN_col", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide14, "nora_wt_r10k_loops_CBS2c_divide_Ren_WN_col", Cm(14.55), Cm(0.05), Cm(10.85), Cm(0.5))

nora_wt_r10k_loops_CBS2c_divide_Nora_WN_Ren = "coolpuppy_pileup_divide_plot_for_Nora_WT_r10k_loops_CBS2c.merge_6comparation.exp_col.png"
# slide15
slide15 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
# pic = slide.shapes.add_picture(png_files[pi-1], left, top, width, height)
slide15.shapes.add_picture(nora_wt_r10k_loops_CBS2c_divide_Nora_WN_Ren, Cm(0), Cm(0.79), Cm(25.4), Cm(12.7))
# 插入文本注释
add_text_anno(slide15, "nora_wt_r10k_loops_CBS2c_divide_Nora_WN_Ren", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))

# 保存 PPT
my_ppt.save('8.3_coolpuppy_pileup_plot_for_Nora_WT_r10k_loop_anchors.use_Nora_WN_Ren_mcool.r10k.summary.pptx')


# 移动相关结果到特定文件夹

In [ ]:
%%bash
mkdir coolpuppy_pileup_plot_for_Nora_WT_r10k_loop_anchors_use_e5_r10k
mv coolpuppy_*for_Nora_WT_r10k_loops*.pdf ./coolpuppy_pileup_plot_for_Nora_WT_r10k_loop_anchors_use_e5_r10k/
mv coolpuppy_*for_Nora_WT_r10k_loops*.png ./coolpuppy_pileup_plot_for_Nora_WT_r10k_loop_anchors_use_e5_r10k/
